In [2]:
# data gather through tweepy API
import pandas as pd
import time
import json
import csv
from tqdm import tqdm
import tweepy
import requests
import os
import numpy as np
api = pd.read_csv(r'C:\Users\63014\Desktop\twitter key.txt')
consumer_key        = api.iloc[0,1]
consumer_secret     = api.iloc[1,1]
access_token        = api.iloc[3,1]
access_token_secret = api.iloc[4,1]
bearer_token        = api.iloc[2,1]

In [3]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
client = tweepy.Client(bearer_token=bearer_token)

In [9]:
df = pd.read_csv('../../data/NAACL_SRW_2016.csv')
df.columns = ['user_id','label']
# 有重复的号码
df = df.drop_duplicates(subset='user_id',keep='first').reset_index(drop=True)
df = df.sample(frac=0.2)
df.head()
print(df.shape)
print(np.unique(df.label))

(3370, 2)
['none' 'racism' 'sexism']


In [5]:
def tqdm_enumerate(iter):
    i = 0
    for y in tqdm(iter):
        yield i, y
        i += 1

In [10]:
delete = []
text_list = []
hashtags_list = []
id_list = []
location_list = []
followers_count_list = []
friends_count_list = []
country_list = []
retweet_count_list = []
favorite_count_list = []
flag = True
for count, i in tqdm_enumerate(iter=df.user_id[::-1]):
    try:
        status = api.get_status(id=str(i))._json
        count += 1
    except:
        delete.append(i)
        continue
    else:
        text = status['text']
        hashtags = status['entities']['hashtags']
        if hashtags.__len__() == 0:
            hashtext = np.NaN
        else:
            hashtext = hashtags[0]['text']
        id = status['user']['id']
        location = status['user']['location']
        followers_count = status['user']['followers_count']
        friends_count = status['user']['friends_count']
        try:
            country = status['place']['country']
        except:
            country = np.NaN
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        text_list.append(text)
        hashtags_list.append(hashtext)
        id_list.append(id)
        location_list.append(location)
        followers_count_list.append(followers_count)
        friends_count_list.append(friends_count)
        country_list.append(country)
        retweet_count_list.append(retweet_count)
        favorite_count_list.append(favorite_count)
    if count % 25 == 0:
        time.sleep(1)
        print('we need to sleep to avoid reaching limit')
    elif count % 100 == 0:
        time.sleep(5)
        print('we need to sleep more to avoid reaching limit')
    elif count % 500 == 0:
        df_temp = pd.DataFrame.from_records({'user_id': id_list, 'text': text_list, 'hashtag': hashtags_list, 'location': location_list, 'follower_count': followers_count_list,
                                                'friends_count': friends_count_list, 'country': country_list, 'retweet_count': retweet_count_list, 'favorite_count': favorite_count_list})
        df_temp.to_csv('./hate_speech_test2.csv', 'a')


  1%|          | 26/3370 [00:10<26:39,  2.09it/s] 

we need to sleep to avoid reaching limit


  1%|▏         | 50/3370 [00:19<29:11,  1.90it/s]  

we need to sleep to avoid reaching limit


  2%|▏         | 75/3370 [00:29<29:13,  1.88it/s]  

...

 97%|█████████▋| 3276/3370 [47:36<00:36,  2.57it/s]

we need to sleep to avoid reaching limit


 98%|█████████▊| 3301/3370 [47:44<00:47,  1.45it/s]

we need to sleep to avoid reaching limit


100%|██████████| 3370/3370 [48:00<00:00,  1.17it/s]


In [14]:
count

3370

In [22]:
# df_clean.loc[:,'id'] = id_list
# df_clean.loc[:,'text'] = text_list
# df_clean.loc[:,'hashtag'] = hashtags_list
# df_clean.loc[:,'location'] = location_list
# df_clean.loc[:,'follower_count'] = followers_count_list
# df_clean.loc[:,'friends_count'] = friends_count_list
# df_clean.loc[:,'country'] = country_list
# df_clean.loc[:,'retweet_count'] = retweet_count_list
# df_clean.loc[:,'favorite_count'] = favorite_count_list

0

In [15]:
# with open('hate_speech.csv','a') as fd:
#     for count,i in tqdm_enumerate(iter=df.user_id):
#         try:
#             status = api.get_status(id=str(i))._json
#         except:
#             continue
#         else:
#             text = status['text']
#             hashtags = status['entities']['hashtags']
#             if hashtags.__len__()==0:
#                 hashtext = np.NaN
#             else:
#                 hashtext = hashtags[0]['text']
#             id = status['user']['id']
#             location = status['user']['location']
#             followers_count = status['user']['followers_count']
#             friends_count = status['user']['friends_count']
#             try:
#                 country = status['place']['country']
#             except:
#                 country = np.NaN
#             retweet_count = status['retweet_count']
#             favorite_count = status['favorite_count']
#             df_temp = pd.DataFrame.from_records()


,user_id,label,text
0,562874396155850752,none,RT @randi_ebooks: Twitter just switched to mon...
1,470393182505545728,none,Because if Elliot Rodger had killed just ONE w...
4,567937595390300160,none,@Grummz @rxptor @Spacekatgal @TheQuinnspiracy ...
8,575948706039492609,none,My face every time I hear or see Kat....hope i...
10,537647038649741312,none,Empathy is the social skill that informs all o...
...,...,...,...
2521,572330892972068864,none,Beans on toast for dinner tonight. Would score...
2522,564099641604861952,sexism,Is it lost on most #QuestionsForMen was essent...
2523,566067783457591297,none,Step 1: post to facebook.\nStep 2: KiA links t...
2524,571499576152231936,sexism,RT @fruitondabottom: @MT8_9 Didn't you hear ab...


In [47]:
df_clean = df.query('user_id not in @delete').copy()
df_clean.loc[:,'id'] = id_list
df_clean.loc[:,'text'] = text_list
df_clean.loc[:,'hashtag'] = hashtags_list
df_clean.loc[:,'location'] = location_list
df_clean.loc[:,'follower_count'] = followers_count_list
df_clean.loc[:,'friends_count'] = friends_count_list
df_clean.loc[:,'country'] = country_list
df_clean.loc[:,'retweet_count'] = retweet_count_list
df_clean.loc[:,'favorite_count'] = favorite_count_list

In [48]:
df_clean.sample(n=5)

,user_id,label,id,text,hashtag,location,follower_count,friends_count,country,retweet_count,favorite_count
4164,414469743911763968,sexism,930620467,RT @ZachDuffy72: I guess #Tammy is the next te...,Tammy,,18807,263,NaN,1,0
15765,501228143609659392,none,719655337,#MKR because 5+ minute add breaks are just the...,MKR,Brisbane,134,414,NaN,1,1
7990,575974815749689344,none,13857342,@undergroundbby I know you're going through a ...,NaN,"Seattle, WA",20161,780,NaN,0,1
16051,537645922461564928,none,20051851,I would have thought the sassy sisters would l...,mkr,"Canberra, Australia",608,1129,NaN,0,2
15686,489515198198194176,none,2949949244,Deconstructed Apple pie...bwilliant #mkr,mkr,"Dunwich, Queensland",68,156,NaN,0,1


In [19]:
df_clean.to_csv('./hate_speech2.csv')

In [23]:
df1 = pd.read_csv('./hate_speech.csv',index_col=0)

In [50]:
df_clean.drop(['user_id','location'],axis=1,inplace=True)

In [51]:
df_clean

,label,id,text,hashtag,follower_count,friends_count,country,retweet_count,favorite_count
6113,none,13857342,"@GemmaNoon it's what i'm *trying* to do, at le...",NaN,20161,780,NaN,0,0
6387,none,13857342,Just saw this fly by on my stats display a few...,NaN,20161,780,NaN,0,4
3289,sexism,82522377,"Sauce, capsicum and stock from a packet / jar....",Mkr,4,64,NaN,0,1
11426,none,13857342,I think I'm just going to play video games all...,NaN,20161,780,NaN,1,30
15331,none,930620467,@ThelmaSleaze We can fight against systemic pr...,NaN,18807,263,NaN,0,0
...,...,...,...,...,...,...,...,...,...
16202,none,2756873076,@LostSailorNY @AJM9588 @CFLancop So true.,NaN,1793,1858,NaN,0,1
10483,none,13857342,@apfergus i'm reading about it now. :),NaN,20161,780,United States,0,0
2090,sexism,2756873076,.@SopranoJM ...can't relate. Have to be over...,NaN,1793,1858,NaN,0,1
10203,none,2522242938,#mkr great catwalk. 10/10,mkr,8,0,NaN,0,0


In [30]:
df_ = df_clean.append(df1)

C:\Users\63014\AppData\Local\Temp\ipykernel_22072\1665766160.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ = df_clean.append(df1)


In [60]:
df_.to_csv('./hate_speech_longer.csv')

In [59]:
df_.drop('country',axis=1,inplace=True)
df_.reset_index(drop=True,inplace=True)

In [62]:
df_.label.unique()

array(['none', 'sexism', 'racism'], dtype=object)